# Alignment hypergraph to variant graph

Convert alignment hypergraph to variant graph before visualization.

(Alignment hypergraph creates a very large dot object, which frustrates visualization. Converting it to a variant graph before visualization reduces the size substantially, at least in normal cases.)

## Initialize

In [1]:
%load_ext autoreload
%autoreload 2
%load_ext memory_profiler

import gc # garbage collection
import operator # comparison operators lt and gt
from collections import defaultdict, deque
from typing import Set, List
from dataclasses import dataclass

import pprint
pp = pprint.PrettyPrinter(indent=2)

## Load data

In [2]:
%run create-blocks.ipynb # data will be in prioritized_blocks

## Create data structures

1. Create witness_node (dataclass) for witness nodes. Properties (not all may be needed):
    1. token string: str
    1. witness: str
    1. offset in witness: int
    1. hyperedge: hyperedge instance (initially null)
1. Create alignment_hyperedge (dataclass) for … wait for it! … alignment hyperedges! Properties:
    1. witness_nodes: set of witness_node objects
1. Create lists for witness_node and alignment_hyperedge instances
1. Create START and END nodes and associated hyperedges

In [3]:
@dataclass(unsafe_hash=True)
class WitnessNode:
    """Write a docstring some day"""
    token_string: str
    witness: str
    witness_offset: int
    hyperedge: hyperedge = None

    def __repr__(self):
        return self.token_string

In [4]:
class AlignmentHyperedge():
    """Write a real docstring someday"""
    def __init__(self, kwargs):
        self.__dict__.update(kwargs) # overwrite the real value
        self._sigla = [key for key in kwargs.keys()]
    def __repr__(self):
        return "|".join(
            [":".join(
                    [str(key), str(getattr(self, key))]
                ) for key in self._sigla]
        )
    def __setitem__(self, key, value):
        if key in self._sigla:
            return
            #raise Exception("Duplicate key exception: "+str(key)+" on object: "+str(self))
        self._sigla.append(key)
        self.__dict__[key] = value
    def __getitem__(self, key):
        return self.__dict__[key]
    def __contains__(self, key):
        return key in self.__dict__
    def values(self):
        return [self[key] for key in self._sigla]
    def items(self):
        return [(key, self.__dict__[key]) for key in self._sigla]

In [5]:
# Store all witness nodes in dictionary of lists
witness_node_lists = defaultdict(list) # one per witness

In [6]:
# create START node, nodes for real tokens, and END node
for index, witness_siglum in enumerate(witness_sigla):
    witness_node_lists[witness_siglum].append(WitnessNode('START', witness_siglum, -1, None))
    for witness_token_offset, witness_token in enumerate(witnesses[index]): # list of tokens in a single witness
        witness_node_lists[witness_siglum].append(WitnessNode(witness_token, witness_siglum, witness_token_offset, None))
    witness_node_lists[witness_siglum].append(WitnessNode('END', witness_siglum, len(token_array), None))

In [7]:
# create hyperedges for START and END
# hyperedges = []
offsets = [0, -1]
for offset in offsets:
    dictionary = {siglum: witness_node_lists[siglum][offset] for siglum in witness_sigla}
    hyperedge = AlignmentHyperedge(dictionary)
    for witness_node in dictionary.values():
        witness_node.hyperedge = hyperedge
#    hyperedges.append(hyperedge)

In [8]:
def create_hyperedges(block):
    """Returns hyperedge data for block, together with offset_start_positions"""
    new_hyperedges = []
    for token_offset_in_block in range(selected_block.token_count):
        # first add the token_offset_in_block to the block start positions
        offset_start_positions = [value + token_offset_in_block for value in selected_block.all_start_positions]
        tokens_to_place = [
            (
                witness_sigla[token_membership_array[token_array_index]],
                token_witness_offset_array[token_array_index]
            )
         for token_array_index in offset_start_positions
#         if not placed_token_bitarray[token_array_index] # already placed, so don’t place it again
        ]
        dictionary = { # data for new hyperedge
            siglum: witness_node_lists[siglum][offset + 1]
            for siglum, offset
            in tokens_to_place
        }
        hyperedge = AlignmentHyperedge(dictionary)
        new_hyperedges.append(hyperedge)

    return (offset_start_positions, new_hyperedges)

In [9]:
def find_floor(siglum, new_offset):
    """Closest hyperedge to the left for a single witness"""
#     print("new_offset: "+ str(new_offset))
    for offset in range(new_offset - 1, -1, -1):
#         print("traversing: "+str(offset))
        # walk backwards looking for node with hyperedge
        node = witness_node_lists[siglum][offset]
#         print("Witness Node: "+str(node))
        if node.hyperedge:
#             print("hyperedge: "+str(node.hyperedge))
            return witness_node_lists[siglum][offset].hyperedge
    raise Exception("We should not get here!")

In [10]:
def find_ceiling(siglum, new_offset):
    """Closest hyperedge to the right for a single witness"""
    for offset in range(new_offset + 1, len(token_array)):
        # walk forwards looking for node with hyperedge
        if witness_node_lists[siglum][offset].hyperedge:
            return witness_node_lists[siglum][offset].hyperedge
    raise Exception("We should not get here!")

## Prose statement of transposition detection

1. Find floor and ceiling for block, where both floor and ceiling are sets of hyperedges, with each member of the set selected on the basis of just one witness.
1. For each hyperedge in the floor, verify that all nodes that belong to that hyperedge are to the left of the leftmost node for that witness in the block we are trying to place. If so, there is no transposition. If that is not the case, at least one witness would be transposed, so do nothing.
1. For each hyperedge in the ceiling, verify that all nodes that belong to that hyperedge are to the right of th rightmost node for that witness in the block we are trying to place. If so, there is no transposition. If that is not the case, at least one witness would be transposed, so do nothing.

In [11]:
def check_for_crossing_hyperedges(
    hyperedge_to_test: AlignmentHyperedge,
    boundary_hyperedges: List[AlignmentHyperedge], # upper or lower
    lower_or_higher: str) -> bool: # last parameter must be 'floor' or 'ceiling'
    """ Check inside floor and ceiling for consistency

    * Is every floor hyperedge to the left of every leftmost block node?
    * And similarly for the ceiling and the right edge?

    Same function checks 'floor' and 'ceiling' (raise on wrong argument)
    Return True iff no crossing hyperedges, i.e., no transposition
    Relies on operator module for gt and lt; syntax is operator.lt(a, b)

    """
    # Set comparison operator to lt or gt
    if lower_or_higher == 'floor':
        comp = operator.lt
    elif lower_or_higher == "ceiling":
        comp = operator.gt
    else:
        raise Exception("Third argument to check_for_crossing_hyperedges() must be 'floor' or 'ceiling'")

    for siglum, witness_node in hyperedge_to_test.items():
        for boundary_hyperedge in boundary_hyperedges:
            if siglum not in boundary_hyperedge._sigla:
                continue
            elif not comp(boundary_hyperedge[siglum].witness_offset, witness_node.witness_offset):
                return False;
    return True # if we're still alive at the end of the function

In [12]:
def check_for_transposition(hyperedges_to_check: List[AlignmentHyperedge]) -> bool:
    """Check for transposition

    List of hyperedges is consecutive because tokens are consecutive in blocks
    Floor of first and ceiling of last are boundaries for the entire list
    """
    first_hyperedge = hyperedges_to_check[0]
    last_hyperedge = hyperedges_to_check[-1]
    # print("\n")
    # print(f"{first_hyperedge=}")
    # print(f"{last_hyperedge=}")
    # Find first hyperedge to left for each witness
    #   and determine whether there is a transposition.
    #   They may not be the same hyperedge (!)
    lower_hyperedges = set()
    for siglum, witness_node in first_hyperedge.items():
        # position of node for each witness in first_hyperedge
        # convert from token offset to node offset
        lower_hyperedges.add(find_floor(siglum, witness_node.witness_offset +1))
    # Find ceiling for all witnesses in second hyperedge
    upper_hyperedges = set()
    for siglum, witness_node in last_hyperedge.items():
        # position of node for each witness in last_hyperedge
        # convert from token offset to node offset
        upper_hyperedges.add(find_ceiling(siglum, witness_node.witness_offset +1))
    # we now have: lower_hyperedges, upper_hyperedges
    # Check inside floor and ceiling for consistency; is every floor hyperedge to the left
    #   of every leftmost block node, and similarly for the ceiling and the right edge?
    floor_ok = check_for_crossing_hyperedges(first_hyperedge, lower_hyperedges, 'floor')
    ceiling_ok = check_for_crossing_hyperedges(last_hyperedge, upper_hyperedges, 'ceiling')
    return floor_ok and ceiling_ok # returns true only if both edges show no transposition

## Augmenting an existing hyperedge

* Every witness node always contains zero or one hyperedge. If zero, create new. If one, extend.
* Provisionally, check for two or more and raise an exception. Our assumption (okay, hope) is that it won't happen.

In [13]:
def merge_hyperedge(potential_hyperedge):
            # check whether a witness_node has got the hyperedge property set
            # a hyperedge may contain zero, some, or all witnesses; we need to handle all these cases
            found = set([witnessnode.hyperedge for siglum, witnessnode in potential_hyperedge.items() if witnessnode.hyperedge])
            if len(found) > 1:
                # found two or more existing hyperedges...
                # remove them from the hyperedges set
#                 for existing_hyperedge in found:
#                     hyperedges.remove(existing_hyperedge)
                # replace them by a new hyperedge
#                 hyperedges.append(potential_hyperedge)
                # update hyperedge property on the witness nodes involved in the hyperedge
                new_hyperedge = potential_hyperedge
                for witness_node in new_hyperedge.values():
                    witness_node.hyperedge = new_hyperedge

#                 print(found) # debug only
#                 raise Exception('Eek! Nodes belong to different existing hyperedges!')
            elif len(found) == 0:
                # we checked for transpositions and we filtered out hyperedges for witness nodes that are already placed
#                 hyperedges.append(potential_hyperedge)
                # update hyperedge property on the witness nodes involved in the hyperedge
                new_hyperedge = potential_hyperedge
                for witness_node in new_hyperedge.values():
                    witness_node.hyperedge = new_hyperedge
            else: # update single existing hyperedge
                for siglum in potential_hyperedge._sigla:
                    list(found)[0][siglum] = potential_hyperedge[siglum] # add node to hyperedge (possibly redundantly)
                    potential_hyperedge[siglum].hyperedge = list(found)[0] # and update hyperedge property of node
#

In [14]:
# create all hyperedges (gingerly)
selected_blocks = filter(lambda x: len(x.all_start_positions) == x.witness_count, prioritized_blocks)
for selected_block in selected_blocks:
    # %memit pass
    offset_start_positions, new_hyperedges = create_hyperedges(selected_block)
    no_transposition = check_for_transposition(new_hyperedges)
    if no_transposition: # all-or-nothing
        for potential_hyperedge in new_hyperedges:
            merge_hyperedge(potential_hyperedge)

## Reduce memory use

In [15]:
del selected_blocks
del prioritized_blocks
del raw_data_dict
del lcp_array
del suffix_array
del token_array
del token_membership_array
del token_to_block_dict
del token_witness_offset_array
del witnesses
gc.collect()

9

## Create variant graph from alignment hypergraph

First create variant graph nodes. Walk over witness nodes in `witness_node_lists`. Keep track of which hyperedges have been added to variant graph. For each witness node:

1. If node has no hyperedge, create variant graph node with selected information from witness node on it.
1. If node has new hyperedge, create variant graph node with selected information from all witness nodes from hyperedge on it.
1. If node has hyperedge that is already on variant graph, do nothing.

Witness nodes contain token string, a witness siglum, a witness offset, and an optional hyperedge. The hyperedge contains, as dictionary items, all witness sigla as keys with witness nodes as values. From this information, we put onto the variant graph node the token string plus the hyperedge dictionary with sigla as keys and witness offsets as values.

**Notes:**

* The variant graph does not need to contain the token string, since we can look it up later, but for ease of visualization we include it here.
* In Real Life Collatex a variant graph node contains properties other than the token string. We ignore that structure in our experiment as we concentrate on visualization.

In [16]:
class VG_node():
    """Variant graph node"""
    def __init__(self, token_string, data): # dictionary of siglum:witness_offset
        self.token_string = token_string
        self.__dict__.update(**data)
        self._sigla = [key for key in data.keys()]
    def __repr__(self):
        return self.token_string + "~" + "|".join([":".join([str(key), str(getattr(self, key))]) for key in self.sigla()])
    def __setitem__(self, key, value):
        self._sigla.append(key)
        self.__dict__[key] = value
    def __getitem__(self, key):
        return self.__dict__[key]
    def __contains__(self, key):
        return key in self.__dict__
    def sigla(self):
        return self._sigla
    def values(self):
        return [self[key] for key in self._sigla]
    def items(self):
        return [(key, self.__dict__[key]) for key in self._sigla]

In [17]:
# create empty variant graph as sets of nodes and edges

VG = {}
VG['nodes'] = set()
VG['edges'] = defaultdict(lambda: defaultdict(set))

# keep track of which nodes (and therefore also hyperedges) have been added
from bitarray import bitarray
VG_tracking = {}
for siglum, witness_node_list in witness_node_lists.items():
    VG_tracking[siglum] = bitarray(len(witness_node_list) - 2)
    VG_tracking[siglum].setall(0)

# map from witness node to variant graph node (needed to construct edges)
# values will be added as we create variant graph nodes
witness_offset_to_VG_node = {}
for siglum in witness_node_lists.keys():
    witness_offset_to_VG_node[siglum] = [None] * (len(witness_node_lists[siglum]) - 2)

In [18]:
# Create variant graph nodes
# witness_node_lists maps from siglum to list of witness node objects
for siglum, witness_node_list in witness_node_lists.items(): # witness by witness
    for witness_node in witness_node_list:
        if not witness_node.token_string in ("START", "END"): # create separately
            if not witness_node.hyperedge:
                # If it doesn't have a hyperedge, create a new variant graph node and add data from witness node
                new_VG_node = VG_node(
                        witness_node.token_string,
                        {witness_node.witness: witness_node.witness_offset}
                    )
                VG['nodes'].add(new_VG_node)
                VG_tracking[siglum][witness_node.witness_offset] = 1 # update bitarray to show that witness node has been added
                witness_offset_to_VG_node[siglum][witness_node.witness_offset] = new_VG_node # add pointer from witness note to VG node
            elif VG_tracking[siglum][witness_node.witness_offset] == 1:
                # If it does have a hyperedge and the hyperedge has already been created, do nothing
                continue
            else:
                # Otherwise it has a new hyperedge, so:
                # 1. create it with empty dictionary
                # 2. add witness nodes from hyperedge nodes to new VG node dictionary
                # 3. update bitarray
                # 4. update pointers from witness node to new VG node
                new_VG_node = VG_node(
                        witness_node.token_string,
                        {}
                    )
                VG['nodes'].add(new_VG_node)
                for hyperedge_node in witness_node.hyperedge.values():
                    new_VG_node[hyperedge_node.witness] = hyperedge_node.witness_offset
                    VG_tracking[hyperedge_node.witness][hyperedge_node.witness_offset] = 1
                    witness_offset_to_VG_node[hyperedge_node.witness][hyperedge_node.witness_offset] = new_VG_node

In [19]:
# Create variant graph edges
for siglum, witness_node_list in witness_node_lists.items():
    edge_pairs = zip(witness_node_list[1:-2], witness_node_list[2:]) # skip START and END; create separately
    for source, target in edge_pairs:
        VG['edges'] \
        [witness_offset_to_VG_node[siglum][source.witness_offset]] \
        [witness_offset_to_VG_node[siglum][target.witness_offset]].add(siglum)

In [20]:
# recover some memory
del witness_node_list
del witness_offset_to_VG_node
del VG_tracking
gc.collect()

0

## Visualize variant graph

Since edges are a set, there should be no duplicates

In [22]:
import graphviz
from IPython.display import SVG
from collections import defaultdict

# node id values must be strings for graphviz
a = graphviz.Digraph(format="svg")
a.attr(rankdir = "LR")
a.attr(rank = 'same')
a.attr(compound='true')

# create START and END nodes
a.node('START')
a.node('END')

# plot nodes, building {node: id} for lookup
node_to_id = {}
for index, node in enumerate(VG['nodes']):
    node_id = str(index)
    node_to_id[node] = node_id
    node_text = node.token_string + " (" + node_id + ")"
    a.node(node_id, label=node_text)

# plot edges
for source_node, inner_dicts in VG['edges'].items():
    for target_node, edge_sigla in inner_dicts.items():
        label = "(all)" if len(edge_sigla) == len(witness_sigla) else ",".join(sorted(edge_sigla))
        a.edge(node_to_id[source_node], node_to_id[target_node], label=label)

# plot START edges
# for siglum, token_list in witness_offset_to_VG_node.items():
#     target_node = witness_offset_to_VG_node[siglum][0]
#     target = node_to_id[target_node]
#     # print(f"{target_node.sigla()=}")
#     if not target in plotted_edges['START']:
#         a.edge('START', target, label=",".join(sorted(target_node.sigla())))
#         plotted_edges['START'].add(target)

# plot END edges
# for siglum, token_list in witness_offset_to_VG_node.items():
#     source_node = witness_offset_to_VG_node[siglum][-1]
#     source = node_to_id[source_node]
#     # print(f"{source_node.sigla()=}")
#     if not target in plotted_edges[source]:
#         a.edge(source, 'END', label=",".join(sorted(source_node.sigla())))
#         plotted_edges[source].add(target)

SVG(a.view())

In [27]:
VG['edges']

defaultdict(<function __main__.<lambda>()>,
            {WHEN~w1:0|w2:0|w3:3|w0:0|w4:3|w5:3: defaultdict(set,
                         {we~w1:1|w2:1|w3:4|w0:1|w4:4|w5:4: {'w0',
                           'w1',
                           'w2',
                           'w3',
                           'w4',
                           'w5'}}),
             we~w1:1|w2:1|w3:4|w0:1|w4:4|w5:4: defaultdict(set,
                         {look~w1:2|w2:2|w3:5|w0:2: {'w0', 'w1', 'w2', 'w3'},
                          compare~w4:5|w5:5: {'w4', 'w5'}}),
             look~w1:2|w2:2|w3:5|w0:2: defaultdict(set,
                         {to~w1:3|w2:3|w3:6|w0:3: {'w0', 'w1', 'w2', 'w3'}}),
             to~w1:3|w2:3|w3:6|w0:3: defaultdict(set,
                         {the~w1:4|w2:4|w3:7|w0:4|w4:6|w5:6: {'w0',
                           'w1',
                           'w2',
                           'w3'}}),
             the~w1:4|w2:4|w3:7|w0:4|w4:6|w5:6: defaultdict(set,
                         {ind

In [0]:
'''
 This function joins the variant graph in place.
 This function is a straight port of the Java version of CollateX.
    :type graph: VariantGraph
 TODO: add transposition support!
'''


def join(graph):
    processed = set()
    end = graph.end
    queue = deque()
    for (_, neighbor) in graph.out_edges(graph.start):
        queue.appendleft(neighbor)
    while queue:
        vertex = queue.popleft()
        out_edges = graph.out_edges(vertex)
        if len(out_edges) is 1:
            (_, join_candidate) = next(iter(out_edges))
            can_join = join_candidate != end and len(graph.in_edges(join_candidate)) == 1
            if can_join:
                join_vertex_and_join_candidate(graph, join_candidate, vertex)
                for (_, neighbor, data) in list(graph.out_edges(join_candidate, data=True)):
                    graph.remove_edge(join_candidate, neighbor)
                    graph.connect(vertex, neighbor, data["label"])
                graph.remove_edge(vertex, join_candidate)
                graph.remove_node(join_candidate)
                queue.appendleft(vertex)
                continue
        processed.add(vertex)
        for (_, neighbor) in out_edges:
            # FIXME: Why do we run out of memory in some cases here, if this is not checked?
            if neighbor not in processed:
                queue.appendleft(neighbor)


def join_vertex_and_join_candidate(graph, join_candidate, vertex):
    # Note: since there is no normalized/non normalized content in the graph
    # a space character is added here for non punctuation tokens

    if re.match(r'^\W', join_candidate.label):
        vertex.label += join_candidate.label
    else:
        vertex.label += (" " + join_candidate.label)
    # join_candidate must have exactly one token (inside a list); left item may have more
    for siglum, token in join_candidate.tokens.items():
        vertex.add_token(siglum, token[0])
